In [1]:
import pandas as pd
import folium
import requests
from io import StringIO

In [2]:
# Get FAA's list of commercial service airports (NPIAS)
npias_url = "https://www.faa.gov/sites/faa.gov/files/2022-09/NPIAS-2023-2027-Appendix-A.xlsx"

header_names = ["State", "City", "Airport", "LocID", "Ownership", "SvcLvl", 
                "Hub", "Role", "Enplaned", "BasedAircraft", "DevelopmentEstimate2023-2027"]
npias_df = pd.read_excel(npias_url, sheet_name=0, header=0, names=header_names)
npias_df

c:\Users\ASUS\miniconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,State,City,Airport,LocID,Ownership,SvcLvl,Hub,Role,Enplaned,BasedAircraft,DevelopmentEstimate2023-2027
0,AK,Adak Island,Adak,ADK,PU,GA,NaN,Basic,2205,0,29283628
1,AK,Akhiok,Akhiok,AKK,PU,GA,NaN,Basic,529,0,10996500
2,AK,Akiachak,Akiachak,Z13,NaN,GA,NaN,Basic,1535,0,5347640
3,AK,Akiak,Akiak,AKI,PU,GA,NaN,Basic,1103,0,6166245
4,AK,Akutan,Akutan,7AK,PU,GA,NaN,Basic,1275,0,1504250
...,...,...,...,...,...,...,...,...,...,...,...
3282,CA,Weed,Weed,O46,PU,GA,NaN,Basic,0,13,3371344
3283,CA,Willits,Ells Field/Willits Municipal,O28,PU,GA,NaN,Basic,0,12,849852
3284,CA,Willows,Willows/Glenn County,WLW,PU,GA,NaN,Local,0,31,1584500
3285,CA,Woodlake,Woodlake,O42,PU,GA,NaN,Basic,0,11,2165445


Check the meaning of the abbreviation in `SvcLvl` in 
https://www.faa.gov/sites/faa.gov/files/2022-10/ARP-NPIAS-2023-Appendix-A.pdf

In [3]:
display(npias_df["SvcLvl"].unique())
display(npias_df["Role"].unique())

array(['GA', 'P', 'CS', 'R'], dtype=object)

array(['Basic', nan, 'Local', 'Unclassified', 'Regional', 'National'],
      dtype=object)

P - Commercial Service - Primary  
CS - Commercial Service - Nonprimary  
R - Reliever Airport  
GA - General Aviation Airport

In [4]:
# Extract valid airport codes (FAA/IATA 3-letter codes)
# NPIAS uses "LocID" which is typically the IATA code for commercial airports

commercial_codes = set(npias_df[
  (npias_df['SvcLvl'].isin(['P'])) 
]["LocID"].dropna().astype(str).str.strip().str.upper())

commercial_codes

{'ABE',
 'ABI',
 'ABQ',
 'ABR',
 'ABY',
 'ACK',
 'ACT',
 'ACV',
 'ACY',
 'ADQ',
 'AEX',
 'AGS',
 'AKN',
 'AKW',
 'ALB',
 'ALO',
 'ALS',
 'ALW',
 'AMA',
 'ANC',
 'APN',
 'ART',
 'ASE',
 'ATL',
 'ATW',
 'ATY',
 'AUS',
 'AVL',
 'AVP',
 'AZO',
 'BBG',
 'BDL',
 'BET',
 'BFF',
 'BFI',
 'BFL',
 'BGM',
 'BGR',
 'BHM',
 'BID',
 'BIL',
 'BIS',
 'BJI',
 'BLI',
 'BLV',
 'BMI',
 'BNA',
 'BOI',
 'BOS',
 'BPT',
 'BQK',
 'BQN',
 'BRD',
 'BRO',
 'BRW',
 'BTM',
 'BTR',
 'BTV',
 'BUF',
 'BUR',
 'BVU',
 'BWI',
 'BZN',
 'CAE',
 'CAK',
 'CDC',
 'CDV',
 'CHA',
 'CHO',
 'CHS',
 'CID',
 'CIU',
 'CKB',
 'CLE',
 'CLL',
 'CLT',
 'CMH',
 'CMI',
 'CMX',
 'CNY',
 'COD',
 'COS',
 'COU',
 'CPR',
 'CPX',
 'CRP',
 'CRW',
 'CSG',
 'CVG',
 'CWA',
 'DAB',
 'DAL',
 'DAY',
 'DBQ',
 'DCA',
 'DEN',
 'DFW',
 'DHN',
 'DIK',
 'DLG',
 'DLH',
 'DRO',
 'DRT',
 'DSM',
 'DTW',
 'DUT',
 'EAR',
 'EAT',
 'EAU',
 'ECP',
 'EGE',
 'EKO',
 'ELM',
 'ELP',
 'ENA',
 'ERI',
 'ESC',
 'EUG',
 'EVV',
 'EWN',
 'EWR',
 'EYW',
 'FAI',
 'FAR',
 'FAT',


In [5]:
# Load OpenFlights airport data (with coordinates)
airport_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
columns = [
    "Airport ID", "Name", "City", "Country", "IATA", "ICAO",
    "Latitude", "Longitude", "Altitude", "Timezone", "DST",
    "Tz database time zone", "Type", "Source"
]

airports_df = pd.read_csv(airport_url, names=columns)
airports_df

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298,272,\N,\N,\N,airport,OurAirports
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


In [6]:
# Filter for USA and valid commercial airports
us_airports = airports_df[
  (airports_df["Country"] == "United States") &
  (airports_df["IATA"] != "\\N") &
  (airports_df["IATA"].isin(commercial_codes))
].copy()

# Clean data
us_airports["Latitude"] = pd.to_numeric(us_airports["Latitude"], errors="coerce")
us_airports["Longitude"] = pd.to_numeric(us_airports["Longitude"], errors="coerce")
us_airports = us_airports.dropna(subset=["Latitude", "Longitude"])

us_airports

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
3216,3415,Hilo International Airport,Hilo,United States,ITO,PHTO,19.721399,-155.048004,38,-10,N,Pacific/Honolulu,airport,OurAirports
3238,3437,Fort Smith Regional Airport,Fort Smith,United States,FSM,KFSM,35.336601,-94.367401,469,-6,A,America/Chicago,airport,OurAirports
3239,3438,Merrill Field,Anchorage,United States,MRI,PAMR,61.213501,-149.843994,137,-9,A,America/Anchorage,airport,OurAirports
3243,3442,Grand Forks International Airport,Grand Forks,United States,GFK,KGFK,47.949299,-97.176102,845,-6,A,America/Chicago,airport,OurAirports
3247,3446,Ernest A. Love Field,Prescott,United States,PRC,KPRC,34.654499,-112.419998,5045,-7,A,America/Phoenix,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5738,7669,Northwest Florida Beaches International Airport,Panama City,United States,ECP,KECP,30.357106,-85.795414,69,-6,A,America/Chicago,airport,OurAirports
6004,8194,Westerly State Airport,Washington County,United States,WST,KWST,41.349602,-71.803398,81,-5,A,America/New_York,airport,OurAirports
6005,8195,Block Island State Airport,Block Island,United States,BID,KBID,41.168098,-71.577797,108,-5,A,America/New_York,airport,OurAirports
6602,9543,Ogden Hinckley Airport,Ogden,United States,OGD,KOGD,41.195900,-112.012001,4473,-7,A,America/Denver,airport,OurAirports


In [7]:
# Generate interactive map centered on the USA

m = folium.Map(
  location=[39.8283, -98.5795], # Approximate center of USA
  zoom_start=4,
  tiles="CartoDB positron"    # Clean, light map style
)

# Add markers for each airport
for _, row in us_airports.iterrows():
  folium.CircleMarker(
    location=[row["Latitude"], row["Longitude"]],
    radius=3, 
    color="#2c7bb6",
    fill=True,
    fill_color="#2c7bb6",
    popup=f"<b>{row['Name']}</b><br>{row['City']}, {row['Country']}",
    tooltip=f"{row['IATA']}: {row['Name']}"
  ).add_to(m)

m